# Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import random as rd
import os
from tqdm import tqdm
import tensorflow_addons as tfa
import pandas as pd
import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar


plt.style.use(['science','ieee'])

os.chdir('/home/mathis/Stage M1/')

2022-06-17 11:54:12.548562: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-17 11:54:12.548589: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Oracle

In [2]:
#GeV [min,max]

tanB_range = [2, 60]
mu_range = [-10**4, 10**4]
M1_range = M2_range = M3_range = [50, 5000]
mA_range = [50, 5000]

mQ_ii_range = mu_ii_range = md_ii_range = mL_ii_range = me_ii_range = [10**2,10**7]
ae_33_range = [-10**4,10**4]
au_ii_range = ad_ii_range = [-10**4,10**4]
mQ_23_range = [0,10**3]
md_23_range = [0,10**3]
au_ij_range = ad_ij_range = [-100,100]

NMFV_range_big= [tanB_range, mu_range, M1_range, M2_range, M3_range, mA_range, mQ_ii_range,
                 mQ_ii_range, mQ_ii_range, mu_ii_range, mu_ii_range, mu_ii_range, md_ii_range,
                 md_ii_range, md_ii_range, mL_ii_range, mL_ii_range, mL_ii_range, me_ii_range,
                 me_ii_range, me_ii_range, ae_33_range, au_ii_range, au_ii_range, au_ii_range,
                 ad_ii_range, ad_ii_range, ad_ii_range, mQ_23_range, md_23_range, au_ij_range,
                 au_ij_range, au_ij_range, au_ij_range, au_ij_range, au_ij_range, ad_ij_range,
                 ad_ij_range, ad_ij_range, ad_ij_range, ad_ij_range, ad_ij_range]


NMFV_min = np.array([i[0] for i in NMFV_range_big])
NMFV_max = np.array([i[1] for i in NMFV_range_big])

NMFV_sub = np.subtract(NMFV_max, NMFV_min)

NMFV_range = [NMFV_min,NMFV_max,NMFV_sub]

In [3]:
def write_slha(NMFV, nom_slha):
    f = open(nom_slha, "w")
    
    f.write("BLOCK MODSEL \n")
    f.write(" 1    0                               #MSSM \n")
    f.write(" 6    1                               #flavour violation \n")
                                                    
    f.write("BLOCK SMINPUTS \n")                    
    f.write("  1   1.27931417e+02                  # alpha^(-1) SM MSbar(M1) \n")
    f.write("  2   1.663900000e-05                 # G_Fermi \n")
    f.write("  3   0.119                           # alphas(MS) SM MSbar \n")
    f.write("  4   91.1876                         # MZ(pole) \n")
    f.write("  5   4.18                            # mb(mb) SM MSbar \n")
    f.write("  6   172.9                           # mtop(pole) \n")
    f.write("  7   1.776860000e+00                 # m mtau(pole) \n")
                                                      
    f.write("BLOCK MINPAR \n")                     
    f.write("  3   "+str(NMFV[0])+"                # tanb \n")
    f.write("  4   1                               # sign mu \n")
                                                   
    f.write("BLOCK EXTPAR \n")                    
    f.write("  0    "+str(-1.0)+"                  # Q \n")
    f.write("  1    "+str(NMFV[2])+"               # M1 \n")
    f.write("  2    "+str(NMFV[3])+"               # M2 \n")
    f.write("  3    "+str(NMFV[4])+"               # M3 \n")
    f.write("  23   "+str(NMFV[1])+"               # mu \n")
    f.write("  26   "+str(NMFV[5])+"               # mA \n")
    f.write("  25   "+str(NMFV[0])+"               # tanb \n")
    
    f.write("BLOCK MSQ2IN \n")                            
    f.write("  1  1    "+str(NMFV[6])+"            # mq2_11 \n")
    f.write("  2  1    "+str(0)+"                  # mq2_12 \n")
    f.write("  3  1    "+str(0)+"                  # mq2_13 \n")
    f.write("  1  2    "+str(0)+"                  # mq2_21 \n")
    f.write("  2  2    "+str(NMFV[7])+"            # mq2_22 \n")
    f.write("  3  2    "+str(0)+"                  # mq2_23 \n")
    f.write("  1  3    "+str(0)+"                  # mq2_31 \n")
    f.write("  2  3    "+str(NMFV[28])+"           # mq2_32 \n")
    f.write("  3  3    "+str(NMFV[8])+"            # mq2_33 \n")
    
    f.write("BLOCK MSU2IN \n")                            
    f.write("  1  1    "+str(NMFV[9])+"            # mu2_11 \n")
    f.write("  2  1    "+str(0)+"                  # mu2_12 \n")
    f.write("  3  1    "+str(0)+"                  # mu2_13 \n")
    f.write("  1  2    "+str(0)+"                  # mu2_21 \n")
    f.write("  2  2    "+str(NMFV[10])+"           # mu2_22 \n")
    f.write("  3  2    "+str(0)+"                  # mu2_23 \n")
    f.write("  1  3    "+str(0)+"                  # mu2_31 \n")
    f.write("  2  3    "+str(0)+"                  # mu2_32 \n")
    f.write("  3  3    "+str(NMFV[11])+"           # mu2_33 \n")
    
    f.write("BLOCK MSD2IN \n")                            
    f.write("  1  1    "+str(NMFV[12])+"           # md2_11 \n")
    f.write("  2  1    "+str(0)+"                  # md2_12 \n")
    f.write("  3  1    "+str(0)+"                  # md2_13 \n")
    f.write("  1  2    "+str(0)+"                  # md2_21 \n")
    f.write("  2  2    "+str(NMFV[13])+"           # md2_22 \n")
    f.write("  3  2    "+str(0)+"                  # md2_23 \n")
    f.write("  1  3    "+str(0)+"                  # md2_31 \n")
    f.write("  2  3    "+str(NMFV[29])+"           # md2_32 \n")
    f.write("  3  3    "+str(NMFV[14])+"           # md2_33 \n")
    
    f.write("BLOCK MSL2IN \n")                            
    f.write("  1  1    "+str(NMFV[15])+"           # ml2_11 \n")
    f.write("  2  1    "+str(0)+"                  # ml2_12 \n")
    f.write("  3  1    "+str(0)+"                  # ml2_13 \n")
    f.write("  1  2    "+str(0)+"                  # ml2_21 \n")
    f.write("  2  2    "+str(NMFV[16])+"           # ml2_22 \n")
    f.write("  3  2    "+str(0)+"                  # ml2_23 \n")
    f.write("  1  3    "+str(0)+"                  # ml2_31 \n")
    f.write("  2  3    "+str(0)+"                  # ml2_32 \n")
    f.write("  3  3    "+str(NMFV[17])+"           # ml2_33 \n")
    
    f.write("BLOCK MSE2IN \n")                            
    f.write("  1  1    "+str(NMFV[18])+"           # me2_11 \n")
    f.write("  2  1    "+str(0)+"                  # me2_12 \n")
    f.write("  3  1    "+str(0)+"                  # me2_13 \n")
    f.write("  1  2    "+str(0)+"                  # me2_21 \n")
    f.write("  2  2    "+str(NMFV[19])+"           # me2_22 \n")
    f.write("  3  2    "+str(0)+"                  # me2_23 \n")
    f.write("  1  3    "+str(0)+"                  # me2_31 \n")
    f.write("  2  3    "+str(0)+"                  # me2_32 \n")
    f.write("  3  3    "+str(NMFV[20])+"           # me2_33 \n")
    
    f.write("BLOCK TEIN \n")                            
    f.write("  1  1    "+str(0)+"                  # T_e_11 \n")
    f.write("  2  1    "+str(0)+"                  # T_e_21 \n")
    f.write("  3  1    "+str(0)+"                  # T_e_31 \n")
    f.write("  1  2    "+str(0)+"                  # T_e_12 \n")
    f.write("  2  2    "+str(0)+"                  # T_e_22 \n")
    f.write("  3  2    "+str(0)+"                  # T_e_32 \n")
    f.write("  1  3    "+str(0)+"                  # T_e_13 \n")
    f.write("  2  3    "+str(0)+"                  # T_e_23 \n")
    f.write("  3  3    "+str(NMFV[21])+"           # T_e_33 \n")
    
    f.write("BLOCK TUIN \n")                            
    f.write("  1  1    "+str(NMFV[22])+"           # T_u_11 \n")
    f.write("  2  1    "+str(NMFV[30])+"           # T_u_21 \n")
    f.write("  3  1    "+str(NMFV[31])+"           # T_u_31 \n")
    f.write("  1  2    "+str(NMFV[32])+"           # T_u_12 \n")
    f.write("  2  2    "+str(NMFV[23])+"           # T_u_22 \n")
    f.write("  3  2    "+str(NMFV[33])+"           # T_u_32 \n")
    f.write("  1  3    "+str(NMFV[34])+"           # T_u_13 \n")
    f.write("  2  3    "+str(NMFV[35])+"           # T_u_23 \n")
    f.write("  3  3    "+str(NMFV[24])+"           # T_u_33 \n")
    
    f.write("BLOCK TDIN \n")                            
    f.write("  1  1    "+str(NMFV[25])+"           # T_d_11 \n")
    f.write("  2  1    "+str(NMFV[36])+"           # T_d_21 \n")
    f.write("  3  1    "+str(NMFV[37])+"           # T_d_31 \n")
    f.write("  1  2    "+str(NMFV[38])+"           # T_d_12 \n")
    f.write("  2  2    "+str(NMFV[26])+"           # T_d_22 \n")
    f.write("  3  2    "+str(NMFV[39])+"           # T_d_32 \n")
    f.write("  1  3    "+str(NMFV[40])+"           # T_d_13 \n")
    f.write("  2  3    "+str(NMFV[41])+"           # T_d_23 \n")
    f.write("  3  3    "+str(NMFV[27])+"           # T_d_33 \n")
    
    f.close()

In [4]:
def Oracle(logits, NMFV_range, ratio, succes_path):
    
    NMFV_AL = np.add(np.multiply(logits,NMFV_range[2]),NMFV_range[0])
    
    write_slha(NMFV_AL, "Advanced NN/NMFV/nmfv_al.in")    
    
    os.system('./SUSY/softsusy-4.1.9/softpoint.x leshouches < Advanced\ NN/NMFV/nmfv_al.in > Advanced\ NN/NMFV/nmfv_al.out')
            
          
    if os.path.getsize('/home/mathis/Stage M1/Advanced NN/NMFV/nmfv_al.out') == 0 :
        ratio[0]+=1
        return 0
        
    with open('Advanced NN/NMFV/nmfv_al.out') as slha_out:
        content = slha_out.read()
        if 'SOFTSUSY problem' in content:
            ratio[0]+=1
            return 0
        if 'LSP # Warning' in content:
            ratio[0]+=1
            return 0
    
    
    
    ratio[1]+=1
    commande = 'mv Advanced\ NN/NMFV/nmfv_al.out '
    nom_fichier = 'nmfv_al_succes_'+str(ratio[1])+'.out'
    os.system(commande+succes_path+nom_fichier)
    return 1   

# Model

In [5]:
path = 'Advanced NN/NMFV/NMFV_run/'
folder_r = 'GAN_2022-06-17-11:47:51'

model = tf.keras.models.load_model(path+folder_r+'/generator.h5')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 25)]              0         
                                                                 
 dense (Dense)               (None, 100)               2600      
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 42)                4242      
                                                                 
Total params: 27,042
Trainable params: 27,042
Non-trainable params: 0
_________________________________________________________________


2022-06-17 11:54:15.374336: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-17 11:54:15.374363: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-17 11:54:15.374383: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (MPC): /proc/driver/nvidia/version does not exist
2022-06-17 11:54:15.374573: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
taille_test = 25000
dataset_test = [[rd.uniform(0,1) for i in range(25)] for j in range(taille_test)]
dataset_test = tf.convert_to_tensor(dataset_test)

logits = model(dataset_test)

In [8]:
os.system('rm -r Advanced\ NN/NMFV/softsusy_succes/')
os.system('mkdir -p Advanced\ NN/NMFV/softsusy_succes')
succes_path = 'Advanced\ NN/NMFV/softsusy_succes/'

ratio = [0,0]
pred = []
for i in tqdm(logits):
            prediction = Oracle(i, NMFV_range, ratio, succes_path)
            pred.append(prediction)

100%|█████████████████████████████████████| 25000/25000 [21:16<00:00, 19.59it/s]


# Récupération data

In [9]:
def recuperation_slha(folder,filename,sus_mass,param,ms2_param,t_param,succes):
    warning = 0
    for j in range(succes):
        file_name =folder+filename+str(j+1)+'.out' 
        
        with open(file_name) as file:
            for indice, ligne in enumerate(file):
                if 'tanb' in ligne:
                    indice_tanb = indice
                
                if 'M_1(MX)' in ligne:
                    indices_M = [indice+i for i in range(3)]
                if 'mu(MX)' in ligne:
                    indice_mu = indice
                if 'mA(pole)' in ligne:
                    indice_mA = indice
                    
                if 'MW' in ligne:
                    indices_masses = [indice+i for i in range(33)]
                    
                if 'input (mhat_Q^2)_{1,1}' in ligne:
                    indices_msq2 = [indice, indice+3, indice+4, indice+5]
                if 'input (mhat_u^2)_{1,1}' in ligne:
                    indices_msu2 = [indice, indice+3, indice+5]
                if 'input (mhat_d^2)_{1,1}' in ligne:
                    indices_msd2 = [indice, indice+3, indice+4, indice+5]
                if 'Block MSL2IN' in ligne:
                    indices_msl2 = [indice+1, indice+4, indice+6]
                if 'Block MSE2IN' in ligne:
                    indices_mse2 = [indice+1, indice+4, indice+6]
                    
                if 'input (That_U)_{1,1}' in ligne:
                    indices_tu = [indice+i for i in range(9)]
                if 'input (That_D)_{1,1}' in ligne:
                    indices_td = [indice+i for i in range(9)]
                if 'input (That_E)_{3,3}' in ligne:
                    indice_te = indice
        
        
        file = open(file_name)
        lignes = file.readlines()
        
        
        param[0].append(float(lignes[indice_tanb].split()[1]))
        
        for step, i in enumerate(param[1]):
            i.append(float(lignes[indices_M[step]].split()[1]))
            
        param[2].append(float(lignes[indice_mu].split()[1]))
        
        param[3].append(float(lignes[indice_mA].split()[1]))
            
        
            
        
        sus_mass[0].append(float(lignes[indices_masses[0]].split()[1]))
        
        for step, i in enumerate(sus_mass[1]):
            i.append(float(lignes[indices_masses[step+1]].split()[1]))
        
        sus_mass[2].append(float(lignes[indices_masses[5]].split()[1]))
        
        sus_mass[3][0].append(float(lignes[indices_masses[6]].split()[1]))
        sus_mass[3][1].append(float(lignes[indices_masses[7]].split()[1]))
        sus_mass[3][2].append(float(lignes[indices_masses[9]].split()[1]))
        sus_mass[3][3].append(float(lignes[indices_masses[10]].split()[1]))
        
        sus_mass[4][0].append(float(lignes[indices_masses[8]].split()[1]))
        sus_mass[4][1].append(float(lignes[indices_masses[11]].split()[1]))
        
        for step, i in enumerate(sus_mass[5]):
            i.append(float(lignes[indices_masses[step+12]].split()[1]))
        
        for step, i in enumerate(sus_mass[6]):
            i.append(float(lignes[indices_masses[step+18]].split()[1]))

        for step, i in enumerate(sus_mass[7]):
            i.append(float(lignes[indices_masses[step+21]].split()[1]))        
        
        for step, i in enumerate(sus_mass[8]):
            i.append(float(lignes[indices_masses[step+27]].split()[1]))
            
            
            
        for step, i in enumerate(ms2_param[0]):
            i.append(float(lignes[indices_msq2[step]].split()[2]))
        for step, i in enumerate(ms2_param[1]):
            i.append(float(lignes[indices_msu2[step]].split()[2]))    
        for step, i in enumerate(ms2_param[2]):
            i.append(float(lignes[indices_msd2[step]].split()[2]))
        for step, i in enumerate(ms2_param[3]):
            i.append(float(lignes[indices_msl2[step]].split()[2]))
        for step, i in enumerate(ms2_param[4]):
            i.append(float(lignes[indices_mse2[step]].split()[2])) 
            
            
            
        for step, i in enumerate(t_param[0]):
            i.append(float(lignes[indices_tu[step]].split()[2]))
        for step, i in enumerate(t_param[1]):
            i.append(float(lignes[indices_td[step]].split()[2])) 
        t_param[2].append(float(lignes[indice_te].split()[2]))
    
    
    
        with open(file_name) as file:
            if 'LSP # Warning' in file.read():
                warning+=1
    return warning

In [10]:
MW_mass = []
higgs_mass = [[] for i in range(4)]        #[h0 , H0 , A0 , H+]
g_mass = []
neutralino_mass = [[] for i in range(4)]   #[N1 , N2 , N3 , N4]
chargino_mass = [[] for i in range(2)]     #[neutralino1 , neutralino2]     
e_mass = [[] for i in range(6)]
nu_mass = [[] for i in range(3)]
d_mass = [[] for i in range(6)]
u_mass = [[] for i in range(6)]

sus_mass = [MW_mass,higgs_mass,g_mass,neutralino_mass,chargino_mass,e_mass,nu_mass,d_mass,u_mass]


tanB_param = []
M_param = [[] for i in range(3)]           #[M1 , M2 , M3
mu_param = []
mA_param = []

param = [tanB_param,M_param,mu_param,mA_param]


msq2_param = [[] for i in range(4)]
msu2_param = [[] for i in range(3)]
msd2_param = [[] for i in range(4)]
msl2_param = [[] for i in range(3)]
mse2_param = [[] for i in range(3)]

ms2_param = [msq2_param,msu2_param,msd2_param,msl2_param,mse2_param]


tu_param = [[] for i in range(9)]
td_param = [[] for i in range(9)]
te_param = []

t_param = [tu_param,td_param,te_param]

folder = 'Advanced NN/NMFV/softsusy_succes/'
file_name = 'nmfv_al_succes_'
warning = recuperation_slha(folder,file_name,sus_mass,param,ms2_param,t_param,ratio[1])

print('Nombre warning:',warning)

Nombre warning: 0


# Sauvegarde Run

In [14]:
from datetime import date, datetime

folder_name = folder_r + '_test'
commande = 'mkdir -p Advanced\ NN/NMFV/NMFV_run/'+folder_name
os.system(commande)

with open('Advanced NN/NMFV/NMFV_run/'+folder_name+'/config_test.txt', 'w') as f:
    f.write('Test generator run '+folder_r+'\n \n')
    f.write('Nombre test: '+str(taille_test)+'\n')
    f.write("Nombre d'échecs: "+str(ratio[0])+'\n')
    f.write("Nombre de succes: "+str(ratio[1])+'\n')
    f.write("Nombre Warning LSP: "+str(warning)+'\n')
    f.write('\n')
    model.summary(print_fn=lambda x: f.write(x + '\n'))
    f.write('\n')
    f.write('\n')
    f.write('Range:')
    f.write('tanB: '+str(NMFV_range_big[0])+'\n')
    f.write('mu: '+str(NMFV_range_big[1])+'\n')
    f.write('M1: '+str(NMFV_range_big[2])+'\n')
    f.write('M2: '+str(NMFV_range_big[3])+'\n')
    f.write('M3: '+str(NMFV_range_big[4])+'\n')
    f.write('mA: '+str(NMFV_range_big[5])+'\n')
    
    f.write('mQ2_ii: '+str(mQ_ii_range)+'\n')
    f.write('mQ2_23: '+str(mQ_23_range)+'\n')
    f.write('mu2_ii: '+str(mu_ii_range)+'\n')
    f.write('md2_ii: '+str(md_ii_range)+'\n')
    f.write('md2_23: '+str(md_23_range)+'\n')
    f.write('mL2_ii: '+str(mL_ii_range)+'\n')
    f.write('me2_ii: '+str(me_ii_range)+'\n')
    
    f.write('au_ii: '+str(au_ii_range)+'\n')
    f.write('au_ij: '+str(au_ij_range)+'\n')
    f.write('ad_ii: '+str(ad_ii_range)+'\n')
    f.write('ad_ij: '+str(ad_ij_range)+'\n')
    f.write('ae_33: '+str(ae_33_range)+'\n')
    
    
model.save('Advanced NN/NMFV/NMFV_run/'+folder_name+'/model.h5')

In [16]:
param_file = zip(tanB_param, M_param[0], M_param[1], M_param[2], mu_param, mA_param, 
                 msq2_param[0], msq2_param[1], msq2_param[2], msq2_param[3], msu2_param[0], 
                 msu2_param[1], msu2_param[2], msd2_param[0], msd2_param[1], msd2_param[2], 
                 msd2_param[3], msl2_param[0], msl2_param[1], msl2_param[2], mse2_param[0], 
                 mse2_param[1], mse2_param[2], tu_param[0], tu_param[1], tu_param[2], 
                 tu_param[3], tu_param[4], tu_param[5], tu_param[6], tu_param[7], tu_param[8], 
                 td_param[0], td_param[1], td_param[2], td_param[3], td_param[4], td_param[5], 
                 td_param[6], td_param[7], td_param[8], te_param, MW_mass, higgs_mass[0], 
                 higgs_mass[1], higgs_mass[2], higgs_mass[3], g_mass, neutralino_mass[0], 
                 neutralino_mass[1], neutralino_mass[2], neutralino_mass[3], chargino_mass[0], 
                 chargino_mass[1], e_mass[0], e_mass[1], e_mass[2], e_mass[3], e_mass[4], 
                 e_mass[5], nu_mass[0], nu_mass[1], nu_mass[2], d_mass[0], d_mass[1], 
                 d_mass[2], d_mass[3], d_mass[4], d_mass[5], u_mass[0], u_mass[1], u_mass[2], 
                 u_mass[3], u_mass[4], u_mass[5])

header_param = ['tanb','M1','M2','M3','mu','mA','msq2_11','msq2_22','msq2_23','msq2_33','msu2_11',
         'msu2_22','msu2_33','msd2_11','msd2_22','msd2_23','msd2_33','msl2_11','msl2_22',
         'msl2_33','mse2_11','mse2_22','mse2_33','tu_11','tu_12','tu_13','tu_21','tu_22',
         'tu_23','tu_31','tu_32','tu_33','td_11','td_12','td_13','td_21','td_22',
         'td_23','td_31','td_32','td_33','te_33','W','h0','H0','A0','H+','g','chi01','chi02', 
         'chi03','chi04','chipm1','chipm2','e1','e2','e3','e4','e5','e6','nu1','nu2','nu3',
          'd1','d2','d3','d4','d5','d6','u1','u2','u3','u4','u5','u6']


import csv
with open('Advanced NN/NMFV/NMFV_run/'+folder_name+'/data_GAN_test.csv', 'w') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(header_param)
    writer.writerows(param_file)

In [17]:
print('Statisique du Neural Network')
print("Nombre d'échecs:",ratio[0])
print("Nombre de succes:",ratio[1])
print('Nombre warning:', warning)
print('Éfficacité du AL:',100*ratio[1]/(ratio[0]+ratio[1]),'%')
print('\n')

Statisique du Neural Network
Nombre d'échecs: 25000
Nombre de succes: 0
Nombre warning: 0
Éfficacité du AL: 0.0 %


